# 第七章 型態、方法與介面 - Part 2: 組合與嵌入

本部分將探討 Go 語言的組合機制，包括結構體嵌入和組合設計模式。Go 採用組合而非繼承的設計哲學，這讓程式碼更加靈活和可維護。

## 用內嵌來組合型態

Go 支援結構體嵌入，這是一種組合（composition）的方式來重用程式碼。內嵌允許一個結構體包含另一個結構體或介面，並自動提升被嵌入型態的方法和欄位。

### 結構體嵌入基礎
**嵌入語法**:
```go
type Person struct {
    Name string
    Age  int
}

type Employee struct {
    Person    // 嵌入 Person 結構體
    JobTitle  string
    Salary    float64
}
```

### 嵌入的特性
- **方法提升**: 被嵌入型態的方法自動成為外層型態的方法
- **欄位提升**: 被嵌入型態的欄位可以直接存取
- **多層嵌入**: 支援多層次的嵌入結構
- **命名選擇**: 可以是匿名嵌入或具名嵌入

In [ ]:
package main

import (
    "fmt"
    "time"
)

// 基礎結構體
type Person struct {
    Name      string
    Age       int
    Email     string
    CreatedAt time.Time
}

// Person 的方法
func (p Person) GetFullInfo() string {
    return fmt.Sprintf("%s (%d years old) - %s", p.Name, p.Age, p.Email)
}

func (p Person) IsAdult() bool {
    return p.Age >= 18
}

func (p *Person) UpdateEmail(email string) {
    p.Email = email
}

func (p Person) Greet() string {
    return "Hi, I'm " + p.Name
}

// 嵌入 Person 的結構體
type Employee struct {
    Person              // 匿名嵌入：方法和欄位會提升
    EmployeeID  string
    Department  string
    Salary      float64
    StartDate   time.Time
}

// Employee 的專屬方法
func (e Employee) GetEmployeeInfo() string {
    return fmt.Sprintf("Employee %s: %s in %s (Salary: %.2f)", 
        e.EmployeeID, e.Name, e.Department, e.Salary)
}

func (e Employee) GetYearsOfService() int {
    return time.Now().Year() - e.StartDate.Year()
}

func (e *Employee) PromoteTo(newDepartment string, newSalary float64) {
    e.Department = newDepartment
    e.Salary = newSalary
}

// Student 也嵌入 Person
type Student struct {
    Person                    // 同樣嵌入 Person
    StudentID    string
    School       string
    Grade        int
    GPA          float64
}

// Student 的專屬方法
func (s Student) GetStudentInfo() string {
    return fmt.Sprintf("Student %s: %s at %s (Grade: %d, GPA: %.2f)", 
        s.StudentID, s.Name, s.School, s.Grade, s.GPA)
}

func (s Student) IsHonorStudent() bool {
    return s.GPA >= 3.5
}

// 重寫嵌入型態的方法
func (s Student) Greet() string {
    return fmt.Sprintf("Hi, I'm %s, a student at %s", s.Name, s.School)
}

func main() {
    fmt.Println("=== 基本嵌入示例 ===")
    
    // 創建 Employee
    emp := Employee{
        Person: Person{
            Name:      "Alice Johnson",
            Age:       28,
            Email:     "alice@company.com",
            CreatedAt: time.Now(),
        },
        EmployeeID: "EMP001",
        Department: "Engineering",
        Salary:     85000,
        StartDate:  time.Date(2020, 1, 15, 0, 0, 0, 0, time.UTC),
    }
    
    // 直接存取嵌入型態的欄位
    fmt.Printf("Employee Name: %s\n", emp.Name)  // 直接存取 Person.Name
    fmt.Printf("Employee Age: %d\n", emp.Age)    // 直接存取 Person.Age
    
    // 調用嵌入型態的方法
    fmt.Println(emp.GetFullInfo())  // 調用 Person.GetFullInfo()
    fmt.Printf("Is Adult: %t\n", emp.IsAdult())  // 調用 Person.IsAdult()
    fmt.Println(emp.Greet())  // 調用 Person.Greet()
    
    // 調用 Employee 專屬方法
    fmt.Println(emp.GetEmployeeInfo())
    fmt.Printf("Years of Service: %d\n", emp.GetYearsOfService())
    
    fmt.Println("\n=== 方法重寫示例 ===")
    
    // 創建 Student
    student := Student{
        Person: Person{
            Name:      "Bob Smith",
            Age:       20,
            Email:     "bob@university.edu",
            CreatedAt: time.Now(),
        },
        StudentID: "STU001",
        School:    "MIT",
        Grade:     3,
        GPA:       3.8,
    }
    
    // Student 重寫了 Greet 方法
    fmt.Println(student.Greet())  // 調用 Student.Greet()，不是 Person.Greet()
    fmt.Println(student.GetStudentInfo())
    fmt.Printf("Is Honor Student: %t\n", student.IsHonorStudent())
    
    // 仍然可以調用 Person 的其他方法
    fmt.Printf("Is Adult: %t\n", student.IsAdult())
    fmt.Println(student.GetFullInfo())
    
    fmt.Println("\n=== 指標接收器方法 ===")
    
    // 修改嵌入型態的資料
    fmt.Printf("Original email: %s\n", emp.Email)
    emp.UpdateEmail("alice.johnson@newcompany.com")  // 調用 Person.UpdateEmail
    fmt.Printf("Updated email: %s\n", emp.Email)
    
    // 修改外層型態的資料
    fmt.Printf("Original department: %s, salary: %.2f\n", emp.Department, emp.Salary)
    emp.PromoteTo("Senior Engineering", 95000)
    fmt.Printf("After promotion: %s, salary: %.2f\n", emp.Department, emp.Salary)
    
    fmt.Println("\n=== 明確存取嵌入型態 ===")
    
    // 有時需要明確存取嵌入的型態
    personPart := emp.Person  // 取得 Person 部分
    fmt.Printf("Person part: %+v\n", personPart)
    
    // 直接對嵌入部分調用方法
    fmt.Println(emp.Person.Greet())  // 明確調用 Person.Greet()
    
    fmt.Println("\n=== 嵌入的優勢 ===")
    fmt.Println("1. 程式碼重用：避免重複實現相同的方法")
    fmt.Println("2. 方法提升：嵌入型態的方法自動可用")
    fmt.Println("3. 欄位提升：可以直接存取嵌入型態的欄位")
    fmt.Println("4. 靈活組合：可以組合多個不同的型態")
    fmt.Println("5. 方法重寫：可以覆蓋嵌入型態的方法行為")
}

### 多重嵌入和複雜組合

Go 支援在一個結構體中嵌入多個型態，這提供了更靈活的組合方式。

In [ ]:
package main

import (
    "fmt"
    "time"
)

// 基礎能力結構體
type Identifiable struct {
    ID        string
    CreatedAt time.Time
}

func (i Identifiable) GetID() string {
    return i.ID
}

func (i Identifiable) GetAge() time.Duration {
    return time.Since(i.CreatedAt)
}

type Nameable struct {
    Name        string
    DisplayName string
}

func (n Nameable) GetName() string {
    if n.DisplayName != "" {
        return n.DisplayName
    }
    return n.Name
}

func (n *Nameable) SetDisplayName(displayName string) {
    n.DisplayName = displayName
}

type Contactable struct {
    Email string
    Phone string
}

func (c Contactable) GetContactInfo() string {
    return fmt.Sprintf("Email: %s, Phone: %s", c.Email, c.Phone)
}

func (c Contactable) HasEmail() bool {
    return c.Email != ""
}

func (c Contactable) HasPhone() bool {
    return c.Phone != ""
}

// 多重嵌入：組合多種能力
type User struct {
    Identifiable  // ID 和創建時間
    Nameable      // 名稱相關
    Contactable   // 聯絡方式
    Role          string
    IsActive      bool
}

// User 專屬方法
func (u User) GetFullProfile() string {
    return fmt.Sprintf("User %s [%s]: %s (Role: %s, Active: %t)", 
        u.GetID(), u.GetName(), u.GetContactInfo(), u.Role, u.IsActive)
}

func (u *User) Activate() {
    u.IsActive = true
}

func (u *User) Deactivate() {
    u.IsActive = false
}

// 另一個多重嵌入的例子
type Product struct {
    Identifiable     // ID 和創建時間
    Nameable         // 產品名稱
    Price       float64
    InStock     int
    Category    string
}

func (p Product) GetProductInfo() string {
    return fmt.Sprintf("Product %s [%s]: $%.2f (Stock: %d, Category: %s)", 
        p.GetID(), p.GetName(), p.Price, p.InStock, p.Category)
}

func (p Product) IsAvailable() bool {
    return p.InStock > 0
}

func (p *Product) UpdateStock(quantity int) {
    p.InStock = quantity
}

// 嵌入具名欄位
type Order struct {
    Identifiable
    Customer    User      // 具名嵌入
    Items       []Product // 產品清單
    TotalAmount float64
    Status      string
}

func (o Order) GetOrderSummary() string {
    return fmt.Sprintf("Order %s: Customer %s, Items: %d, Total: $%.2f, Status: %s", 
        o.GetID(), o.Customer.GetName(), len(o.Items), o.TotalAmount, o.Status)
}

func (o *Order) AddItem(product Product) {
    o.Items = append(o.Items, product)
    o.TotalAmount += product.Price
}

func main() {
    fmt.Println("=== 多重嵌入示例 ===")
    
    // 創建具有多重嵌入的 User
    user := User{
        Identifiable: Identifiable{
            ID:        "USER001",
            CreatedAt: time.Now().Add(-24 * time.Hour), // 一天前創建
        },
        Nameable: Nameable{
            Name:        "John Doe",
            DisplayName: "Johnny",
        },
        Contactable: Contactable{
            Email: "john@example.com",
            Phone: "+1-555-0123",
        },
        Role:     "Admin",
        IsActive: true,
    }
    
    // 使用來自不同嵌入型態的方法
    fmt.Printf("User ID: %s\n", user.GetID())              // 來自 Identifiable
    fmt.Printf("User Name: %s\n", user.GetName())          // 來自 Nameable
    fmt.Printf("Contact: %s\n", user.GetContactInfo())     // 來自 Contactable
    fmt.Printf("Account Age: %v\n", user.GetAge().Hours()) // 來自 Identifiable
    
    fmt.Println(user.GetFullProfile())
    
    fmt.Println("\n=== 產品示例 ===")
    
    product1 := Product{
        Identifiable: Identifiable{
            ID:        "PROD001",
            CreatedAt: time.Now(),
        },
        Nameable: Nameable{
            Name:        "Go Programming Book",
            DisplayName: "Go Book",
        },
        Price:    59.99,
        InStock:  50,
        Category: "Books",
    }
    
    product2 := Product{
        Identifiable: Identifiable{
            ID:        "PROD002",
            CreatedAt: time.Now(),
        },
        Nameable: Nameable{
            Name: "Programming Keyboard",
        },
        Price:    149.99,
        InStock:  0,
        Category: "Electronics",
    }
    
    fmt.Println(product1.GetProductInfo())
    fmt.Printf("Available: %t\n", product1.IsAvailable())
    
    fmt.Println(product2.GetProductInfo())
    fmt.Printf("Available: %t\n", product2.IsAvailable())
    
    fmt.Println("\n=== 具名嵌入示例 ===")
    
    order := Order{
        Identifiable: Identifiable{
            ID:        "ORD001",
            CreatedAt: time.Now(),
        },
        Customer: user,  // 具名嵌入
        Status:   "Pending",
    }
    
    // 添加商品到訂單
    order.AddItem(product1)
    
    fmt.Println(order.GetOrderSummary())
    
    // 存取具名嵌入的成員
    fmt.Printf("Customer Email: %s\n", order.Customer.Email)
    fmt.Printf("Customer Contact: %s\n", order.Customer.GetContactInfo())
    
    fmt.Println("\n=== 方法衝突處理 ===")
    
    // 如果多個嵌入型態有同名方法，需要明確指定
    fmt.Printf("User ID (via Identifiable): %s\n", user.Identifiable.GetID())
    fmt.Printf("User Name (via Nameable): %s\n", user.Nameable.GetName())
    
    // 修改嵌入型態的資料
    user.SetDisplayName("John D.")
    fmt.Printf("Updated display name: %s\n", user.GetName())
    
    product1.UpdateStock(45)
    fmt.Printf("Updated stock: %d\n", product1.InStock)
    
    fmt.Println("\n=== 多重嵌入的優勢 ===")
    fmt.Println("1. 能力組合：將不同的能力組合成複雜的型態")
    fmt.Println("2. 程式碼重用：避免在每個型態中重複實現相同功能")
    fmt.Println("3. 關注點分離：每個嵌入型態負責特定的功能")
    fmt.Println("4. 靈活設計：可以選擇性地組合需要的能力")
    fmt.Println("5. 易於維護：修改特定能力只需要修改對應的嵌入型態")
}

## 內嵌不是繼承

重要提醒：Go 的嵌入是組合（composition），不是繼承（inheritance）。這有一些重要的含義和區別。

### 組合 vs. 繼承的關鍵差異

**Go 的組合特性**:
- 被嵌入的型態保持獨立性
- 方法調用時，接收器仍然是原始型態
- 沒有多型的傳統概念
- 可以同時嵌入多個型態

**與傳統繼承的區別**:
- 沒有 `super` 或 `parent` 的概念
- 方法重寫不影響內部方法調用
- 型態轉換需要明確指定
- 更靈活的結構組織

In [ ]:
package main

import "fmt"

// 基礎型態
type Base struct {
    name  string
    value int
}

func (b Base) GetName() string {
    return b.name
}

func (b Base) GetInfo() string {
    return fmt.Sprintf("Base: %s (value: %d)", b.name, b.value)
}

// Base 的方法調用其他方法
func (b Base) Describe() string {
    // 注意：這裡調用的是 Base.GetName()，不是 Derived.GetName()
    return "Description of " + b.GetName()
}

func (b *Base) SetValue(v int) {
    b.value = v
    fmt.Printf("Base.SetValue called: set value to %d\n", v)
}

// 嵌入 Base 的型態
type Derived struct {
    Base        // 嵌入 Base
    extraInfo   string
    multiplier  int
}

// Derived "重寫" Base 的方法
func (d Derived) GetName() string {
    return fmt.Sprintf("%s (Derived)", d.name)
}

func (d Derived) GetInfo() string {
    return fmt.Sprintf("Derived: %s (value: %d, extra: %s)", 
        d.name, d.value*d.multiplier, d.extraInfo)
}

// Derived 的專屬方法
func (d Derived) GetExtraInfo() string {
    return d.extraInfo
}

func (d *Derived) SetMultiplier(m int) {
    d.multiplier = m
}

// 演示方法調用的複雜性
type Logger struct {
    prefix string
}

func (l Logger) log(message string) {
    fmt.Printf("[%s] %s\n", l.prefix, message)
}

func (l Logger) Info(message string) {
    // 內部方法調用：總是調用 Logger.log
    l.log("INFO: " + message)
}

func (l Logger) Error(message string) {
    l.log("ERROR: " + message)
}

type AdvancedLogger struct {
    Logger
    enableColors bool
}

// "重寫" Logger 的 log 方法
func (al AdvancedLogger) log(message string) {
    if al.enableColors {
        fmt.Printf("\033[32m[%s] %s\033[0m\n", al.prefix, message) // 綠色
    } else {
        fmt.Printf("[%s] %s\n", al.prefix, message)
    }
}

// 展示型態轉換
func processBase(b Base) {
    fmt.Printf("Processing Base: %s\n", b.GetInfo())
}

func main() {
    fmt.Println("=== 組合不是繼承的證明 ===")
    
    // 創建實例
    derived := Derived{
        Base: Base{
            name:  "test",
            value: 10,
        },
        extraInfo:  "additional data",
        multiplier: 2,
    }
    
    fmt.Println("--- 方法調用行為 ---")
    
    // 直接調用：使用 Derived 的版本
    fmt.Printf("derived.GetName(): %s\n", derived.GetName())  
    // 輸出: test (Derived)
    
    fmt.Printf("derived.GetInfo(): %s\n", derived.GetInfo())  
    // 輸出: Derived: test (value: 20, extra: additional data)
    
    // 重要：Base.Describe() 內部調用的是 Base.GetName()，不是 Derived.GetName()
    fmt.Printf("derived.Describe(): %s\n", derived.Describe())  
    // 輸出: Description of test  (不是 "Description of test (Derived)")
    
    fmt.Println("\n--- 明確調用嵌入型態的方法 ---")
    
    // 明確調用 Base 的方法
    fmt.Printf("derived.Base.GetName(): %s\n", derived.Base.GetName())  
    // 輸出: test
    
    fmt.Printf("derived.Base.GetInfo(): %s\n", derived.Base.GetInfo())  
    // 輸出: Base: test (value: 10)
    
    fmt.Println("\n--- 型態轉換 ---")
    
    // Derived 不能直接當作 Base 使用（不是繼承關係）
    // processBase(derived)  // 編譯錯誤！
    
    // 需要明確提取 Base 部分
    processBase(derived.Base)  // OK
    
    fmt.Println("\n--- 指標接收器的行為 ---")
    
    fmt.Printf("Original value: %d\n", derived.value)
    derived.SetValue(20)  // 調用 Base.SetValue
    fmt.Printf("After SetValue: %d\n", derived.value)
    
    fmt.Println("\n=== Logger 示例：內部方法調用 ===")
    
    logger := Logger{prefix: "APP"}
    advancedLogger := AdvancedLogger{
        Logger:       Logger{prefix: "ADV"},
        enableColors: true,
    }
    
    fmt.Println("--- 標準 Logger ---")
    logger.Info("This is an info message")
    logger.Error("This is an error message")
    
    fmt.Println("\n--- Advanced Logger ---")
    // 注意：Logger.Info() 內部調用的仍然是 Logger.log()，不是 AdvancedLogger.log()
    advancedLogger.Info("This is an info message")   // 不會有顏色
    advancedLogger.Error("This is an error message") // 不會有顏色
    
    // 直接調用 AdvancedLogger.log() 才會有顏色
    advancedLogger.log("Direct call to AdvancedLogger.log")  // 會有顏色
    
    fmt.Println("\n=== 組合 vs. 繼承對比 ===")
    fmt.Println("組合特性：")
    fmt.Println("1. 嵌入型態保持獨立性")
    fmt.Println("2. 內部方法調用不受外層"重寫"影響")
    fmt.Println("3. 需要明確的型態轉換")
    fmt.Println("4. 可以組合多個型態")
    fmt.Println("5. 更靈活，避免深層繼承鏈")
    
    fmt.Println("\n繼承特性（Go 不支援）：")
    fmt.Println("1. 子類是父類的特殊形式")
    fmt.Println("2. 方法重寫影響所有調用")
    fmt.Println("3. 自動的多型行為")
    fmt.Println("4. is-a 關係")
    fmt.Println("5. 可能導致複雜的繼承層次")
    
    fmt.Println("\n=== 實際應用建議 ===")
    fmt.Println("1. 使用組合來重用程式碼")
    fmt.Println("2. 明確理解方法調用的行為")
    fmt.Println("3. 避免過度依賴方法"重寫"")
    fmt.Println("4. 優先考慮介面而不是嵌入")
    fmt.Println("5. 保持結構簡潔明確")
}

### 嵌入的最佳實踐

理解嵌入不是繼承後，我們來看看使用嵌入的最佳實踐。

In [ ]:
package main

import (
    "fmt"
    "sync"
    "time"
)

// =====================================================================
// 最佳實踐 1: 嵌入用於能力組合，而非資料繼承
// =====================================================================

// 定義各種能力
type Timestamped struct {
    CreatedAt time.Time
    UpdatedAt time.Time
}

func (t *Timestamped) Touch() {
    t.UpdatedAt = time.Now()
}

func (t Timestamped) Age() time.Duration {
    return time.Since(t.CreatedAt)
}

type Lockable struct {
    mu sync.RWMutex
}

func (l *Lockable) Lock() {
    l.mu.Lock()
}

func (l *Lockable) Unlock() {
    l.mu.Unlock()
}

func (l *Lockable) RLock() {
    l.mu.RLock()
}

func (l *Lockable) RUnlock() {
    l.mu.RUnlock()
}

type Validatable struct {
    errors []string
}

func (v *Validatable) AddError(err string) {
    v.errors = append(v.errors, err)
}

func (v *Validatable) IsValid() bool {
    return len(v.errors) == 0
}

func (v *Validatable) GetErrors() []string {
    return v.errors
}

func (v *Validatable) ClearErrors() {
    v.errors = v.errors[:0]
}

// =====================================================================
// 最佳實踐 2: 組合不同能力創建業務實體
// =====================================================================

type User struct {
    Timestamped  // 時間戳能力
    Lockable     // 並發安全能力
    Validatable  // 驗證能力
    
    ID       string
    Name     string
    Email    string
    IsActive bool
}

// User 的業務方法
func NewUser(id, name, email string) *User {
    now := time.Now()
    return &User{
        Timestamped: Timestamped{
            CreatedAt: now,
            UpdatedAt: now,
        },
        ID:       id,
        Name:     name,
        Email:    email,
        IsActive: true,
    }
}

func (u *User) Validate() {
    u.ClearErrors()  // 使用 Validatable 的方法
    
    if u.ID == "" {
        u.AddError("ID is required")
    }
    if u.Name == "" {
        u.AddError("Name is required")
    }
    if u.Email == "" {
        u.AddError("Email is required")
    }
}

func (u *User) UpdateName(name string) {
    u.Lock()         // 使用 Lockable 的方法
    defer u.Unlock()
    
    u.Name = name
    u.Touch()        // 使用 Timestamped 的方法
}

func (u *User) GetInfo() string {
    u.RLock()        // 使用 Lockable 的方法
    defer u.RUnlock()
    
    return fmt.Sprintf("User %s: %s <%s> (Active: %t, Age: %v)", 
        u.ID, u.Name, u.Email, u.IsActive, u.Age()) // 使用 Timestamped 的方法
}

// =====================================================================
// 最佳實踐 3: 避免深度嵌入
// =====================================================================

// 不好的做法：多層嵌入
// type A struct { value int }
// type B struct { A }
// type C struct { B }
// type D struct { C }  // 過於複雜

// 好的做法：扁平化組合
type ConfigurableService struct {
    Timestamped
    Lockable
    
    Name     string
    Config   map[string]interface{}
    IsRunning bool
}

func (s *ConfigurableService) Start() error {
    s.Lock()
    defer s.Unlock()
    
    if s.IsRunning {
        return fmt.Errorf("service %s is already running", s.Name)
    }
    
    s.IsRunning = true
    s.Touch()
    return nil
}

func (s *ConfigurableService) Stop() {
    s.Lock()
    defer s.Unlock()
    
    s.IsRunning = false
    s.Touch()
}

func (s *ConfigurableService) GetStatus() string {
    s.RLock()
    defer s.RUnlock()
    
    status := "stopped"
    if s.IsRunning {
        status = "running"
    }
    
    return fmt.Sprintf("Service %s is %s (last updated: %v)", 
        s.Name, status, s.UpdatedAt.Format("15:04:05"))
}

func main() {
    fmt.Println("=== 嵌入最佳實踐示例 ===")
    
    // 創建用戶
    user := NewUser("USER001", "Alice", "alice@example.com")
    
    fmt.Println("--- 初始狀態 ---")
    fmt.Println(user.GetInfo())
    
    // 驗證用戶
    user.Validate()
    fmt.Printf("Validation result: %t\n", user.IsValid())
    
    // 模擬並發修改
    fmt.Println("\n--- 並發安全修改 ---")
    
    var wg sync.WaitGroup
    
    // 啟動多個 goroutine 同時修改用戶名稱
    for i := 0; i < 3; i++ {
        wg.Add(1)
        go func(index int) {
            defer wg.Done()
            time.Sleep(time.Millisecond * time.Duration(index*100))
            user.UpdateName(fmt.Sprintf("Alice-%d", index))
            fmt.Printf("Goroutine %d: %s\n", index, user.GetInfo())
        }(i)
    }
    
    wg.Wait()
    
    fmt.Println("\n--- 服務示例 ---")
    
    service := &ConfigurableService{
        Timestamped: Timestamped{
            CreatedAt: time.Now(),
            UpdatedAt: time.Now(),
        },
        Name: "WebService",
        Config: map[string]interface{}{
            "port": 8080,
            "debug": true,
        },
    }
    
    fmt.Println(service.GetStatus())
    
    err := service.Start()
    if err != nil {
        fmt.Printf("Failed to start service: %v\n", err)
    } else {
        fmt.Println("Service started successfully")
        fmt.Println(service.GetStatus())
    }
    
    // 嘗試再次啟動
    err = service.Start()
    if err != nil {
        fmt.Printf("Expected error: %v\n", err)
    }
    
    service.Stop()
    fmt.Println(service.GetStatus())
    
    fmt.Println("\n--- 無效用戶示例 ---")
    
    invalidUser := NewUser("", "", "")  // 空值
    invalidUser.Validate()
    
    fmt.Printf("Is valid: %t\n", invalidUser.IsValid())
    fmt.Printf("Errors: %v\n", invalidUser.GetErrors())
    
    fmt.Println("\n=== 最佳實踐總結 ===")
    fmt.Println("1. 嵌入用於能力組合，不是資料繼承")
    fmt.Println("2. 保持嵌入結構扁平化")
    fmt.Println("3. 每個嵌入型態負責單一職責")
    fmt.Println("4. 優先考慮介面而不是嵌入")
    fmt.Println("5. 明確理解方法調用行為")
    fmt.Println("6. 使用具名嵌入避免衝突")
    fmt.Println("7. 組合勝過繼承")
}